<a href="https://colab.research.google.com/github/Shahabshms/Machine_Learning_Solution_1/blob/main/Support_Vector_Machines_with_Gaussian_Kernel_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem 3: Support Vector Machines
For this problem, consider the data set (mystery.data) attached to this homework that, like Problem 1, contains four numeric attributes per row and the fifth entry is the class variable (either $+$ or $-$). Find a perfect classifier for this data set using support vector machines. Your solution should explain the optimization problem that you solved and provide the learned parameters, the optimal margin, and the support vectors. Note, for full credit, your solution should only make use of a quadratic programming solver, i.e., you should not rely on existing SVM toolkits.

In [1]:
import numpy as np
import pandas as pd
import sys, os

col_names = ['x1','x2','x3','x4','y']
!wget -nc https://personal.utdallas.edu/~shahab.shams/files/ML_Spring_2021/mystery.data
data =  pd.read_csv('mystery.data', sep="," , names = col_names)
print(data)

--2021-03-07 17:19:09--  https://personal.utdallas.edu/~shahab.shams/files/ML_Spring_2021/mystery.data
Resolving personal.utdallas.edu (personal.utdallas.edu)... 129.110.182.249
Connecting to personal.utdallas.edu (personal.utdallas.edu)|129.110.182.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34159 (33K)
Saving to: ‘mystery.data’

mystery.data        100%[===================>]  33.36K   104KB/s    in 0.3s    

2021-03-07 17:19:12 (104 KB/s) - ‘mystery.data’ saved [34159/34159]

          x1        x2       x3       x4  y
0    0.14343  0.258500  0.79126  0.53526  1
1    0.19117  0.526940  0.28061  0.77913  1
2    0.40839  0.239850  0.35756  0.80489  1
3    0.16432  0.002530  0.40666  0.89868  1
4    0.27962  0.177280  0.58566  0.73986  1
..       ...       ...      ...      ... ..
995  0.35432  0.235120  0.91744  0.98868 -1
996  1.02650  0.756380  0.53262  0.30093 -1
997  0.46145  0.751200  0.40619  1.02850 -1
998  0.49309  0.076142  0.52661  1.21400 -1

In [2]:
Y = data['y'].values
Y = Y.reshape(-1,1)
print('Y dimention: ',Y.shape)
X = data[['x1','x2','x3','x4']].values
print('X dimnetion: ',X.shape)

Y dimention:  (1000, 1)
X dimnetion:  (1000, 4)


Primal Problem:
$$\min \frac{1}{2}\Vert w \Vert_2^2$$
such that
$$\forall i \in \{1,\dots,M\} \quad\quad  1 - y^{(i)}(w^T x^{(i)} + b) \leq 0. $$
To see why, please check the slides and lectures. 

The promal problem is a quadradic obtimization and we can solve it. However this data set that we have is not linearly separable in its original domain.  To find the perfect classifier, one possible approch is to hire a modified feature vector like $[x_1^2,x_2^2,x_3^2,x_4^2, x_1x_2x_3x_4]$. But here we try to solve the dual problem. 

Lagrangian:

$$L(w,\lambda)  = \frac{1}{2}w^Tw + \sum_{i=1}^M \lambda_i (1-y^{(i)}(w^Tx^{(i)} + b))$$

$$ \frac{\partial L(w,\lambda)}{\partial w} = w - \sum_{i=1}^{M}\lambda_i y^{(i)}x^{(i)} = 0 \rightarrow w = \sum_{i=1}^{M}\lambda_iy^{(i)}x^{(i)}$$
$$ \frac{\partial L(w,\lambda)}{\partial b} =  -\sum_{i=1}^{M}\lambda_iy^{(i)} = 0 \rightarrow \sum_{i=1}^{M}\lambda_iy^{(i)} = 0 $$

Now by substituting $w$ back in the Lagrangian we get the dual problem.

Dual Problem

$$\max_{\lambda} -\frac{1}{2}\sum_{i=1}^{M}\sum_{j=1}^{M} \lambda_i\lambda_jy^{(i)}y^{(j)}\langle x^{(i)},x^{(j)}\rangle + \sum_{i=1}^{M}\lambda_i$$
such that
$$\sum_{i=1}^{M} \lambda_iy^{(i)} = 0$$
$$\lambda_i\geq 0 \quad \forall i\in\{1,\dots,M\}$$

The result of Dual problem is the Lagrange multipliers, $\lambda = [\lambda_1,\dots,\lambda_M]$. These variables show us which data points are support vectors. 
$$\forall i \in \{1,\dots,M\} \text{ if } \lambda_i > 0 \text{ then } x^{(i)} \text{ is a support vector.}$$
To see why, please check the lectures and slides, or come during office hours. 

In the following, we will see how to use support vectors to get a classifier. 

Kernel Trick for Dual SVM

$$\max_{\lambda} -\frac{1}{2}\sum_{i=1}^M\sum_{j=1}^M \lambda_i\lambda_jy^{(i)}y^{(j)}K(x^{(i)},x^{(j)}) + \sum_{i=1}^M \lambda_i$$
The Kernel function can be anything such as Gaussian, $K(x^{(i)},x^{(j)}) = \exp(-\gamma\Vert x^{(i)} - x^{(j)}\Vert^2)$.

In [3]:
import numpy as np

def poly_func(X,poly_case):
  if poly_case == 'Full-2':
    return np.array([X[0],X[1],X[2],X[3],
                     X[0]**2,X[1]**2,X[2]**2,X[3]**2,
                     X[0]*X[1],X[1]*X[2],X[2]*X[3],X[0]*X[2],X[1]*X[3],X[0]*X[3],
                     X[0]*X[1]*X[2]*X[3]])
  elif poly_case == 5:
    return np.array([X[0]**2,X[1]**2,X[2]**2,X[3]**2,X[0]*X[1]*X[2]*X[3]])

  elif poly_case == 'Linear':
    return X

def K(X1,X2,kernel_option,gamma = 1,poly_case = 'Linear'):
  if kernel_option == 'Gaussian':
    return  np.exp(-gamma * np.linalg.norm(X1 - X2)**2)/2
  if kernel_option == 'Polynomial':
    v1 = poly_func(X1,poly_case)
    v2 = poly_func(X2,poly_case)
    return v1.dot(v2)

Solving Quadratic Programming with CVXOPT

$$\min \frac{1}{2}x^T P x + q^Tx$$
such that 
$$Gx \preceq h$$
$$Ax = b$$

See the Quadratic Programming
 from https://cvxopt.org/userguide/coneprog.html


Here, we do not implement the optimization process from the scratch. Instead we utilize CVXOPT. In order to use CVXOPT's API, we have to convert our Dual Problem into a proper notation that is undrestandable for CVXOPT. So, the followin step is just creating new matrices with proper names and then simply passing them to CVXOPT.
Just notice that our variable is $\lambda$ and not $x$. Both $y$ and $x$ are given through the data set, and it's only Lagranle multipliers that we have to find.

The translation into CVXOPT is as follow.

Given $Y\in\mathbb{R}^{M\times 1}$ and $X\in\mathbb{R}^{M\times 4}$
0. Convert the maximization to minimization by multiplying the objectiv function to -1.  
1. $P = YY^T\odot K(X,X)$      
when $\odot$ is the Hadamard product, and matrix $K(X,X)\in\mathbb{R}^{M\times M}$ such that $\forall i,j\in\{1,\dots,M\}$, $K(X,X)_{i,j} = K(X_i,X_j)$. 
2. $q = [-1]^M$
3. $G = -I_{M\times M}$
4. $h = [0]^M$
5. $A = Y^T$
6. $b = 0$ 

In [4]:
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers

def SVM_with_Kernel(X,Y,kernel_option, gamma = 1,poly_case = 'Linear'):
  m,n = X.shape
  H = np.zeros([m,m])
  for i in range(m):
    for j in range(m):
      H[i,j] = Y[i]*Y[j] * K(X[i],X[j], kernel_option,gamma, poly_case)

#Converting into cvxopt format
  P = cvxopt_matrix(H)
  q = cvxopt_matrix(-np.ones((m, 1)))
  G = cvxopt_matrix(-np.eye(m))
  h = cvxopt_matrix(np.zeros(m))
  A = cvxopt_matrix(Y.reshape(1, -1)* 1.)
  b = cvxopt_matrix(np.zeros(1))
#Run solver
  solvers.options['show_progress'] = False
  sol = solvers.qp(P, q, G, h, A, b);
  lambdas = np.array(sol['x']);

  return lambdas

What happen after we have found Lagrange multipliers? 

Looking back at the procedure that converted the Primal problem into the Dual problem, we find 
$$w = \sum_{i=1}^{M}\lambda_iy^{(i)}x^{(i)}.$$
$y$ and $x$ are given. And now that we have $\lambda$, we can simply find $w$. For finding the intercept, $b$, we did not encouter any specific formula. But, since we have the support vectors, we can calculate $b$ as well. Remmember that we started by forcing
$$\forall i \in \{1,\dots,M\} \quad\quad  1 - y^{(i)}(w^T x^{(i)} + b) \leq 0. $$
We know that for support vectors, $1 - y^{(i)}(w^T x^{(i)} + b) = 0$. So, for each support vectors as $s$, we have $$b = \frac{1 - y^{(s)}w^Tx^{(s)}}{y^{(s)}} \rightarrow \begin{cases} b = 1 - w^Tx^{(s)}& y^{(s)} = 1 \\ b = -1 - w^Tx{(s)}& y^{(s)} = -1\end{cases}$$
So,
$$b = y^{(s)} - w^Tx^{(s)}.$$




In [5]:
kernel_option = 'Polynomial'
# 4 is the Linear

dims = {'Linear':4,5:5,'Full-2':15}

for poly_case in ['Linear',5,'Full-2']:
  print('\nKernel Option: ',kernel_option,poly_case)
  lambdas = SVM_with_Kernel(X,Y,kernel_option,poly_case = poly_case)

  modified_X = np.zeros([Y.shape[0],dims[poly_case]])
  for n in range(Y.shape[0]):
    modified_X[n] = poly_func(X[n],poly_case)

  w = ((Y * lambdas).T @ modified_X).reshape(-1,1);
  S = (lambdas > 1e-3).flatten();
  b = Y[S] - np.dot(modified_X[S], w);
  # print('lambdas = ',lambdas[lambdas > 1e-4])
  Y_pred = modified_X.dot(w) + b[0]
  print('w = ', w.flatten())
  print('b = ', b[0])
  res = sum([int(Y[i] == np.sign(Y_pred[i])) for i in range(Y.shape[0])])
  print('Accuracy:', res)
  if poly_case == 'Linear':continue
  print('Support Vectors: \n', np.where(lambdas>0.001)[0])
  print(Y[np.where(lambdas>0.001)[0]].T[0])


Kernel Option:  Polynomial Linear
w =  [-0.00305176 -0.00536728 -0.00656128 -0.00317383]
b =  [1.00871566]
Accuracy: 500

Kernel Option:  Polynomial 5
w =  [-2.00026926e+00 -2.00029657e+00 -2.00027860e+00 -2.00028939e+00
 -1.01471662e-04]
b =  [3.00031234]
Accuracy: 1000
Support Vectors: 
 [  9  64  94 139 144 190 198 213 277 358 411 413 450 468 484 643 645 724
 735 810 888 891]
[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1 -1 -1 -1 -1 -1 -1]

Kernel Option:  Polynomial Full-2
w =  [-0.28567985 -0.29206962 -0.25582616 -0.33389673 -1.94176315 -1.94520011
 -1.9568138  -1.92587606  0.12357456  0.11490359  0.12791295  0.11128354
  0.15288799  0.14159597 -0.04773691]
b =  [3.3360581]
Accuracy: 1000
Support Vectors: 
 [139 328 645 700 746 816 817 833 852 860 879 960]
[ 1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]


For the Gaussian Kernel, predicting is slightly different. Remmember how we get to use the Kernel function.
$$\quad\quad\Phi(x^{(i)})^T\Phi(x^{(j)}) \equiv K(x^{(i)},x^{(j)}) $$
We start by creating the modified version of our data set, applying $\Phi(x)$ to every data point. Furthur, we replace $\Phi(x^{(i)})^T\Phi(x^{(j)})$ with $K(x^{(i)},x^{(j)})$ for simplicity. However, this trick allows us to use Kernel functions that we do not necessarily have a feature vector corresponding to. The Gaussian Kernel is one of those Kernel functions that requires a feature vector of lenght infinity. You may think this makes Gaussian Kernel useless. But actually this is why Gaussian Kernel is the most popular one. The main advantage of the Gaussian Kernel is that, with a very low cost, it takes your data set into an inifinite dimention and you do the classification there. But in the case of SVM, it only results in the support vectors, based on which you can not immediately calculate a separating line. The separating need to be done based on the similarities that each query point has with the support vectors. This task requires the influence that each support vector has in the separating process. 

In [6]:
lambdas = SVM_with_Kernel(X,Y,'Gaussian', gamma = 1/ (X.var() * X.shape[1]))
supports = X[np.where(lambdas > 1)[0]]
print('Indices of support vectors:\n',np.where(lambdas > 1)[0])
print('Support Vectors:\n',supports)

Indices of support vectors:
 [  7  45 134 162 191 224 261 265 342 358 464 591 628 650 700 746 776 817
 833 850 860 879 939 979 994]
Support Vectors:
 [[0.0043834 0.60143   0.78558   0.14537  ]
 [0.65996   0.035779  0.7408    0.11994  ]
 [0.28969   0.19852   0.93181   0.091634 ]
 [0.95018   0.084484  0.016428  0.29957  ]
 [0.057479  0.0093453 0.83091   0.55335  ]
 [0.84726   0.038806  0.52722   0.051884 ]
 [0.99068   0.054552  0.085061  0.091362 ]
 [0.059681  0.96868   0.065506  0.23196  ]
 [0.63954   0.017487  0.036738  0.76768  ]
 [0.60991   0.7887    0.06143   0.046915 ]
 [0.035301  0.07096   0.022447  0.9966   ]
 [0.038325  0.0025055 1.0011    0.99821  ]
 [0.024464  0.056681  0.20296   1.3982   ]
 [0.041857  1.1835    0.079195  0.76902  ]
 [0.1363    0.022961  0.01358   1.4074   ]
 [1.2888    0.02535   0.021734  0.58119  ]
 [0.87856   1.0758    0.098352  0.24741  ]
 [0.2554    0.43854   1.3177    0.078137 ]
 [1.356     0.30684   0.13052   0.22375  ]
 [0.70627   0.042314  0.032845  1

In [7]:
similarities = np.zeros([Y.shape[0],supports.shape[0]])
for m in range(Y.shape[0]):
  for s in range(supports.shape[0]):
    similarities[m,s] = K(X[m],supports[s],kernel_option='Gaussian',gamma=1/ (X.var() * X.shape[1]))

In [8]:
lambdas = SVM_with_Kernel(similarities,Y,'Polynomial',poly_case='Linear')

w = ((Y * lambdas).T @ similarities).reshape(-1,1);
S = (lambdas > 1e-3).flatten();
b = Y[S] - np.dot(similarities[S], w);
Y_pred = similarities.dot(w) + b[0]
res = sum([int(Y[i] == np.sign(Y_pred[i])) for i in range(Y.shape[0])])
print('Accuracy:', res)

Accuracy: 1000


Compare with sklearn.svm

This is not a part of your homework, but it is good to have an alternative method and check the result. 

Please check the support vectors that we have found using the Gaussian Kernel, and compare them with those that sklearn has found.

In [9]:
from sklearn.svm import SVC
print('\nKernel: Radial Basis')
clf = SVC(kernel = 'rbf')
clf.fit(X, Y.ravel()) 
Y_pred = clf.predict(X).reshape(-1,1)
print('accuracy: ',np.sum(Y_pred == Y))
print('Indices of support vectors = \n', np.sort(clf.support_))
# print('Support vectors = \n', clf.support_vectors_)
print('Number of support vectors for each class = ', clf.n_support_)

print('\nKernel: Polynomial')
clf = SVC(kernel = 'poly')
clf.fit(X, Y.ravel()) 
Y_pred = clf.predict(X).reshape(-1,1)
print('accuracy: ',np.sum(Y_pred == Y))
print('Indices of support vectors = \n', np.sort(clf.support_))
# print('Support vectors = \n', clf.support_vectors_)
print('Number of support vectors for each class = ', clf.n_support_)

print('\nKernel: Linear')
clf = SVC(kernel = 'linear')
clf.fit(X, Y.ravel()) 
Y_pred = clf.predict(X).reshape(-1,1)
print('accuracy: ',np.sum(Y_pred == Y))
print('w: ',clf.coef_)
print('b: ',clf.intercept_)





Kernel: Radial Basis
accuracy:  1000
Indices of support vectors = 
 [  7  45  81  84  93 134 147 162 169 191 224 247 259 261 265 270 279 315
 342 343 358 389 409 443 464 491 501 503 560 566 591 593 628 631 641 643
 645 650 659 664 676 700 709 734 746 776 779 789 814 816 817 822 829 833
 850 852 860 879 880 882 906 922 939 979 990 994]
Number of support vectors for each class =  [40 26]

Kernel: Polynomial
accuracy:  1000
Indices of support vectors = 
 [ 56  74  78 117 118 234 253 360 370 463 501 566 591 643 645 700 709 746
 814 817 850 879 979]
Number of support vectors for each class =  [13 10]

Kernel: Linear
accuracy:  977
w:  [[-4.58962092 -4.70044817 -4.3506897  -6.10745074]]
b:  [10.41611276]
